In [9]:
import os
import json

from credentials import OPENAPI_KEY
os.environ["OPENAI_API_KEY"] = OPENAPI_KEY

import openai
from tqdm import tqdm
import pandas as pd


In [10]:
client = openai.OpenAI()

In [11]:
ROOT_PATH = "PATH"
ds_name = "3000tweet"
ds_path = f"{ROOT_PATH}/{ds_name}.jsonl"
with open(ds_path, "r") as f:
    my_dataset = [json.loads(line) for line in f]

In [12]:
my_dataset[0]

{'messages': [{'role': 'system',
   'content': 'VRL-GPT-4o Türkçe metinlerin duygusunu verebilen bir chatbottur. (Pozitif, Nötr, Negatif)'},
  {'role': 'user',
   'content': 'Bu Türkçe metin hangi duyguyu içeriyor?: "dun Turkcelle tepkilerimizden sonra bugün Turkcell twittera sponsor olmuþ. Ne tesadüf deðil mi ? :)"? (Pozitif/Nötr/Negatif)'},
  {'role': 'assistant', 'content': 'Pozitif'}]}

In [32]:
preds = []
refs  = []

for example in tqdm(my_dataset):
    response = client.chat.completions.create(
        model="gpt-4o",  # or "gpt-4-turbo"
        messages=example["messages"][:2]
    )
    
    model_output = response.choices[0].message.content
    ground_truth = example["messages"][-1]["content"]
    preds.append(model_output)
    refs.append(ground_truth)

100%|██████████| 3000/3000 [39:44<00:00,  1.26it/s]  


In [33]:
output_dir = "/cta/users/anajafi/chatGPT/GPT4o/outputs"
output_df = pd.DataFrame({"preds": preds, "refs": refs})
output_df.to_csv(f"{output_dir}/gpt4o-{ds_name}.csv", index=False)

In [35]:
output_df["preds"].value_counts()

preds
Negatif                                                                                                                                                            755
Pozitif.                                                                                                                                                           467
Pozitif                                                                                                                                                            448
Negatif.                                                                                                                                                           435
Nötr                                                                                                                                                               264
                                                                                                                                                               